# Homogeneous Coordinates

## Imports and Setup

In [32]:
%load_ext autoreload
%autoreload 2

import sys
import os

sys.path.append("../")

import logging
from pathlib import Path

from icecream import ic

from IPython.display import display, clear_output
import ipywidgets as wid
from utils.ipywidgets_extended import (
    widgets_styling,
    widgets_styling_slider,
    MultiSelect,
    RadioSelect,
)

from utils.setup_notebook import init_notebook
from utils.setup_logging import setup_logging
import utils.memoize as memoize

init_notebook()
setup_logging("INFO")
memoize.set_file_store_path("homogeneous_coordinates")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
import numpy as np
import pandas as pd
import scipy as sp
import numba as nb
from numba import cuda
import cv2
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

from utils.benchmarking import LogTimer
from utils.plotting_tools import (
    SmartFigure,
    to_ipy_image,
    plot_kernel,
    plot_matrix,
)
from utils.image_tools import load_image, LoadedImage
import utils.dyn_module as dyn
from utils.cv2_tools import draw_keypoints, draw_matches
from utils.distinct_colors import rgbs

logging.getLogger("numba.cuda.cudadrv.driver").setLevel(logging.WARNING)

In [34]:
reset_memoize_store_button = wid.Button(description="Reset memoize store")
reset_memoize_store_button.on_click(lambda x: memoize.reset_store())
display(reset_memoize_store_button)

Button(description='Reset memoize store', style=ButtonStyle())

## Projection Matrix

In [ ]:
projection_smart_figure = SmartFigure(figsize=(12, 6))

KEY_PROJECTION_PROJECTIVE_DIVIDE_TERM = "projection_projective_divide_term"
projection_projective_divide_term_slider = wid.FloatSlider(
    value=memoize.get(KEY_PROJECTION_PROJECTIVE_DIVIDE_TERM, -1.0),
    min=-10.0,
    max=10.0,
    step=0.5,
    continuous_update=False,
    description="Projective Divide Term",
    **widgets_styling_slider,
)

margin = 0.5


def on_projection_menu_change(change=None):
    memoize.set(
        KEY_PROJECTION_PROJECTIVE_DIVIDE_TERM,
        projection_projective_divide_term_slider.value,
    )

    global projection_smart_figure
    # Remove the existing axes
    for ax in projection_smart_figure.fig.get_axes():
        ax.remove()

    # Define a function to plot a cube
    def plot_cube(ax, vertices, color="blue"):
        # Draw the cube defined by its 8 vertices
        faces = [
            [vertices[j] for j in [0, 1, 5, 4]],  # front face
            [vertices[j] for j in [7, 6, 2, 3]],  # back face
            [vertices[j] for j in [0, 3, 7, 4]],  # left face
            [vertices[j] for j in [1, 2, 6, 5]],  # right face
            [vertices[j] for j in [0, 1, 2, 3]],  # bottom face
            [vertices[j] for j in [4, 5, 6, 7]],
        ]  # top face

        # create a poly3dcollection from the faces
        ax.add_collection3d(
            Poly3DCollection(
                faces, facecolors=color, linewidths=1, edgecolors="r", alpha=0.25
            )
        )

        # Plot the vertices with a dot
        color_iter = rgbs()
        for vertex in vertices:
            draw_point(ax, vertex, next(color_iter), size=40)

    def draw_point(ax, point, color="red", size=10):
        ax.scatter(point[0], point[1], point[2], color=color, s=size)

    # Function to apply the 4x4 projection matrix
    def apply_matrix(vertices, matrix):
        transformed_vertices = []
        for vertex in vertices:
            # Convert to homogeneous coordinates (x, y, z, 1)
            homogeneous_vertex = np.append(vertex, 1)
            # Apply the matrix transformation
            transformed_vertex = matrix @ homogeneous_vertex
            # Convert back from homogeneous (x, y, z, w) to (x, y, z)
            transformed_vertex = transformed_vertex[:3] / transformed_vertex[3]
            transformed_vertices.append(transformed_vertex)
        return np.array(transformed_vertices)

    # Define the unit cube vertices. A 2x2x2 cube centered at the origin.
    unit_cube_vertices = np.array(
        [
            [1, 1, 1],  # vertex 0 (front-top-right)
            [1, -1, 1],  # vertex 1 (front-bottom-right)
            [-1, -1, 1],  # vertex 2 (front-bottom-left)
            [-1, 1, 1],  # vertex 3 (front-top-left)
            [1, 1, -1],  # vertex 4 (back-top-right)
            [1, -1, -1],  # vertex 5 (back-bottom-right)
            [-1, -1, -1],  # vertex 6 (back-bottom-left)
            [-1, 1, -1],
        ]
    )  # vertex 7 (back-top-left)

    translation = np.array([0.0, 1.0, 3.0])
    # translate the unit cube by the translation vector using a 4x4 matrix
    translation_matrix = np.eye(4)
    translation_matrix[:3, 3] = translation
    unit_cube_vertices = apply_matrix(unit_cube_vertices, translation_matrix)

    # Define a unit 4x4 matrix (set to identity matrix initially, change these to distort)
    pdt = projection_projective_divide_term_slider.value
    transformation_matrix = np.array(
        [
            [1.0, 0.0, 0.0, 0.0],
            [0.0, 1.0, 0.0, 0.0],
            [0.0, 0.0, 1.0, -1],
            [
                0.0,
                0.0,
                1.0,
                pdt,
            ],
        ]
    )

    # Apply matrix to cube vertices
    transformed_vertices = apply_matrix(unit_cube_vertices, transformation_matrix)

    # Create a 3D plot to display the original and transformed cubes
    fig = projection_smart_figure.fig

    # Subplot 1: Original cube
    ax1 = fig.add_subplot(121, projection="3d")
    plot_cube(ax1, unit_cube_vertices, color="blue")
    ax1.set_title("Original Cube")
    ax1.set_xlabel("X")
    ax1.set_ylabel("Y")
    ax1.set_zlabel("Z")
    ax1.margins(x=margin, y=margin, z=margin)
    ax1.set_box_aspect([1, 1, 1])  # Set equal scaling
    draw_point(ax1, [0, 0, 0], color="black", size=20)

    # Subplot 2: Transformed cube
    ax2 = fig.add_subplot(122, projection="3d")
    plot_cube(ax2, transformed_vertices, color="green")
    ax2.set_title("Transformed (Distorted) Cube")
    ax2.set_xlabel("X")
    ax2.set_ylabel("Y")
    ax2.set_zlabel("Z")
    ax2.margins(x=margin, y=margin, z=margin)
    ax2.set_box_aspect([1, 1, 1])  # Set equal scaling
    draw_point(ax2, [0, 0, 0], color="black", size=20)

    fig.tight_layout()
    # fig.canvas.layout.min_width = "400px"
    # fig.canvas.layout.flex = "1 1 auto"
    # fig.canvas.layout.width = "auto"


projection_projective_divide_term_slider.observe(
    on_projection_menu_change, names="value"
)

display(
    wid.VBox(
        [projection_projective_divide_term_slider, projection_smart_figure.fig.canvas]
    )
)

on_projection_menu_change()

AttributeError: 'NoneType' object has no attribute 'canvas'

AttributeError: 'NoneType' object has no attribute 'canvas'

C:\Users\_\AppData\Local\Temp\ipykernel_4232\3259808144.py:64: RuntimeWarning: divide by zero encountered in divide
  transformed_vertex = transformed_vertex[:3] / transformed_vertex[3]
C:\Users\_\AppData\Local\Temp\ipykernel_4232\3259808144.py:64: RuntimeWarning: invalid value encountered in divide
  transformed_vertex = transformed_vertex[:3] / transformed_vertex[3]
C:\Users\_\AppData\Local\Temp\ipykernel_4232\3259808144.py:64: RuntimeWarning: divide by zero encountered in divide
  transformed_vertex = transformed_vertex[:3] / transformed_vertex[3]
C:\Users\_\AppData\Local\Temp\ipykernel_4232\3259808144.py:64: RuntimeWarning: invalid value encountered in divide
  transformed_vertex = transformed_vertex[:3] / transformed_vertex[3]
